\
# Reinforcement Learning (DQN) Tutorial
**Author**: [Adam Paszke](https://github.com/apaszke)
            [Mark Towers](https://github.com/pseudo-rnd-thoughts)


This tutorial shows how to use PyTorch to train a Deep Q Learning (DQN) agent
on the CartPole-v1 task from [Gymnasium](https://gymnasium.farama.org)_.

**Task**

The agent has to decide between two actions - moving the cart left or
right - so that the pole attached to it stays upright. You can find more
information about the environment and other more challenging environments at
[Gymnasium's website](https://gymnasium.farama.org/environments/classic_control/cart_pole/)_.

.. figure:: /_static/img/cartpole.gif
   :alt: CartPole

   CartPole

As the agent observes the current state of the environment and chooses
an action, the environment *transitions* to a new state, and also
returns a reward that indicates the consequences of the action. In this
task, rewards are +1 for every incremental timestep and the environment
terminates if the pole falls over too far or the cart moves more than 2.4
units away from center. This means better performing scenarios will run
for longer duration, accumulating larger return.

The CartPole task is designed so that the inputs to the agent are 4 real
values representing the environment state (position, velocity, etc.).
We take these 4 inputs without any scaling and pass them through a
small fully-connected network with 2 outputs, one for each action.
The network is trained to predict the expected value for each action,
given the input state. The action with the highest expected value is
then chosen.


**Packages**


First, let's import needed packages. Firstly, we need
[gymnasium](https://gymnasium.farama.org/)_ for the environment,
installed by using `pip`. This is a fork of the original OpenAI
Gym project and maintained by the same team since Gym v0.19.
If you are running this in Google Colab, run:


We'll also use the following from PyTorch:

-  neural networks (``torch.nn``)
-  optimization (``torch.optim``)
-  automatic differentiation (``torch.autograd``)


In [1]:
# %matplotlib inline
# %matplotlib qt
# # !pip install swig
# !pip install minatar==1.0.13
# # !pip install gymnasium
# # !pip install "gymnasium[all]"
# # 

In [2]:
from minatar import Environment
# import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f

# SEED = 42

# # # Set random seed for NumPy
# np.random.seed(SEED)

# # # Set random seed for Python's built-in random module
# random.seed(SEED)


# torch.manual_seed(SEED)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(SEED)
#     torch.cuda.manual_seed_all(SEED)
#     torch.backends.cudnn.deterministic = True
# #     # torch.backends.cudnn.benchmark = False



# env = gym.make(
#     "LunarLander-v2",
#     continuous = False,
#     gravity = -10.0,
#     enable_wind = False,
#     wind_power = 15.0,
#     turbulence_power = 1.5,
# )

# env.seed(SEED)

# env = gym.make('LunarLander-v2')
env = Environment('asterix')

# env = gym.make('CartPole-v1')
# env = gym.make('MountainCar-v0', max_episode_steps=1000)
# env = gym.make('MountainCar-v0', max_episode_steps=500)
# env = gym.make("Acrobot-v1")

# env = gym.make("BipedalWalker-v3")
# env.action_space.seed(SEED)
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Replay Memory

We'll be using experience replay memory for training our DQN. It stores
the transitions that the agent observes, allowing us to reuse this data
later. By sampling from it randomly, the transitions that build up a
batch are decorrelated. It has been shown that this greatly stabilizes
and improves the DQN training procedure.

For this, we're going to need two classes:

-  ``Transition`` - a named tuple representing a single transition in
   our environment. It essentially maps (state, action) pairs
   to their (next_state, reward) result, with the state being the
   screen difference image as described later on.
-  ``ReplayMemory`` - a cyclic buffer of bounded size that holds the
   transitions observed recently. It also implements a ``.sample()``
   method for selecting a random batch of transitions for training.




In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

Now, let's define our model. But first, let's quickly recap what a DQN is.

## DQN algorithm

Our environment is deterministic, so all equations presented here are
also formulated deterministically for the sake of simplicity. In the
reinforcement learning literature, they would also contain expectations
over stochastic transitions in the environment.

Our aim will be to train a policy that tries to maximize the discounted,
cumulative reward
$R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_t$, where
$R_{t_0}$ is also known as the *return*. The discount,
$\gamma$, should be a constant between $0$ and $1$
that ensures the sum converges. A lower $\gamma$ makes
rewards from the uncertain far future less important for our agent
than the ones in the near future that it can be fairly confident
about. It also encourages agents to collect reward closer in time
than equivalent rewards that are temporally far away in the future.

The main idea behind Q-learning is that if we had a function
$Q^*: State \times Action \rightarrow \mathbb{R}$, that could tell
us what our return would be, if we were to take an action in a given
state, then we could easily construct a policy that maximizes our
rewards:

\begin{align}\pi^*(s) = \arg\!\max_a \ Q^*(s, a)\end{align}

However, we don't know everything about the world, so we don't have
access to $Q^*$. But, since neural networks are universal function
approximators, we can simply create one and train it to resemble
$Q^*$.

For our training update rule, we'll use a fact that every $Q$
function for some policy obeys the Bellman equation:

\begin{align}Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))\end{align}

The difference between the two sides of the equality is known as the
temporal difference error, $\delta$:

\begin{align}\delta = Q(s, a) - (r + \gamma \max_a' Q(s', a))\end{align}

To minimize this error, we will use the [Huber
loss](https://en.wikipedia.org/wiki/Huber_loss)_. The Huber loss acts
like the mean squared error when the error is small, but like the mean
absolute error when the error is large - this makes it more robust to
outliers when the estimates of $Q$ are very noisy. We calculate
this over a batch of transitions, $B$, sampled from the replay
memory:

\begin{align}\mathcal{L} = \frac{1}{|B|}\sum_{(s, a, s', r) \ \in \ B} \mathcal{L}(\delta)\end{align}

\begin{align}\text{where} \quad \mathcal{L}(\delta) = \begin{cases}
     \frac{1}{2}{\delta^2}  & \text{for } |\delta| \le 1, \\
     |\delta| - \frac{1}{2} & \text{otherwise.}
   \end{cases}\end{align}

### Q-network

Our model will be a feed forward  neural network that takes in the
difference between the current and previous screen patches. It has two
outputs, representing $Q(s, \mathrm{left})$ and
$Q(s, \mathrm{right})$ (where $s$ is the input to the
network). In effect, the network is trying to predict the *expected return* of
taking each action given the current input.




In [4]:
class DQN(nn.Module):

    def __init__(self, in_channels, n_actions):
        super(DQN, self).__init__()
        # One hidden 2D convolution layer:
        #   in_channels: variable
        #   out_channels: 16
        #   kernel_size: 3 of a 3x3 filter matrix
        #   stride: 1
        self.conv = nn.Conv2d(in_channels, 16, kernel_size=3, stride=1)
        # Final fully connected hidden layer:
        #   the number of linear unit depends on the output of the conv
        #   the output consist 128 rectified units
        def size_linear_unit(size, kernel_size=3, stride=1):
            return (size - (kernel_size - 1) - 1) // stride + 1
        num_linear_units = size_linear_unit(10) * size_linear_unit(10) * 16
        self.fc_hidden = nn.Linear(in_features=num_linear_units, out_features=128)
        
        self.output = nn.Linear(in_features=128, out_features=n_actions)
        
#         self.layer1 = nn.Linear(n_observations, 128)
#         self.layer2 = nn.Linear(128,128)
# #         self.layer3 = nn.Linear(128,128)
#         # self.layer4 = nn.Linear(128,128)
#         self.layer4 = nn.Linear(128, n_actions)
        
        

    # As per implementation instructions according to pytorch, the forward function should be overwritten by all
    # subclasses
    def forward(self, x):
        # Rectified output from the first conv layer
        x = f.relu(self.conv(x))

        # Rectified output from the final hidden layer
        x = f.relu(self.fc_hidden(x.view(x.size(0), -1)))

        # Returns the output from the fully-connected linear layer
        return self.output(x)


    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
#     def forward(self, x):
#         x = F.relu(self.layer1(x))
#         x = F.relu(self.layer2(x))
# #         x = F.relu(self.layer3(x))
#         # x = F.relu(self.layer4(x))
#         return self.layer4(x)

#     def forward_correlation(self, x):
#         x = F.relu(self.layer1(x))
# #         x = F.relu(self.layer2(x))
# #         x = F.relu(self.layer3(x))
#         # x = F.relu(self.layer4(x))
#         return F.relu(self.layer2(x))

    def forward_correlation(self, x):
     # Rectified output from the first conv layer
        x = f.relu(self.conv(x))

        # Rectified output from the final hidden layer
        return f.relu(self.fc_hidden(x.view(x.size(0), -1)))

## Training

### Hyperparameters and utilities
This cell instantiates our model and its optimizer, and defines some
utilities:

-  ``select_action`` - will select an action accordingly to an epsilon
   greedy policy. Simply put, we'll sometimes use our model for choosing
   the action, and sometimes we'll just sample one uniformly. The
   probability of choosing a random action will start at ``EPS_START``
   and will decay exponentially towards ``EPS_END``. ``EPS_DECAY``
   controls the rate of the decay.
-  ``plot_durations`` - a helper for plotting the duration of episodes,
   along with an average over the last 100 episodes (the measure used in
   the official evaluations). The plot will be underneath the cell
   containing the main training loop, and will update after every
   episode.




In [5]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 64
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# Get number of actions from gym action space
n_actions = env.num_actions()
in_channels = env.state_shape()[2]

# Get the number of state observations
# state, info = env.reset()

policy_net = DQN(in_channels, n_actions).to(device)
target_net = DQN(in_channels, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

# optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
optimizer = optim.Adam(policy_net.parameters(), lr = LR)
# faster_lr = 1e-3
# slower_lr = 1e-3

###  CHANGE THISSSSSS
# slower_lr = 1e-3

# # Create separate parameter groups for layers 1-3 and layer 4
# slower_params = list(map(id, policy_net.layer1.parameters() + policy_net.layer2.parameters() + policy_net.layer3.parameters()))
# faster_params = list(map(id, policy_net.layer4.parameters()))

# # Create an optimizer with different learning rates for each parameter group
# optimizer = optim.AdamW(
#     [
#         {"params": [p for p in policy_net.parameters() if id(p) in faster_params], "lr": faster_lr},
#         {"params": [p for p in policy_net.parameters() if id(p) in slower_params], "lr": slower_lr},
#     ]
# )

# params = [
#     {'params': policy_net.layer1.parameters(), 'lr':slower_lr },
#     {'params': policy_net.layer2.parameters(), 'lr': slower_lr},
# #     {'params': policy_net.layer3.parameters(), 'lr': slower_lr},  # Last two layers
#     {'params': policy_net.layer4.parameters(), 'lr': faster_lr},  # Last two layers
# ]

# Define your optimizer with different learning rates for each parameter group
# optimizer = optim.Adam(params)



memory = ReplayMemory(50000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device)


episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

### Training loop

Finally, the code for training our model.

Here, you can find an ``optimize_model`` function that performs a
single step of the optimization. It first samples a batch, concatenates
all the tensors into a single one, computes $Q(s_t, a_t)$ and
$V(s_{t+1}) = \max_a Q(s_{t+1}, a)$, and combines them into our
loss. By definition we set $V(s) = 0$ if $s$ is a terminal
state. We also use a target network to compute $V(s_{t+1})$ for
added stability. The target network is updated at every step with a
[soft update](https://arxiv.org/pdf/1509.02971.pdf)_ controlled by
the hyperparameter ``TAU``, which was previously defined.




K means and then sampling from those clusters


In [6]:
def get_state(s):
    return (torch.tensor(s, device=device).permute(2, 0, 1)).unsqueeze(0).float()

In [7]:

from sklearn.cluster import KMeans

# Create your Gym environment
# env = gym.make('YourEnvNameHere')

# Define the number of clusters (bins)
k = 25
# k=10
# k=50

# Initialize an empty list to store the clustered states
clustered_states = []

# Reset the environment to get the initial state
state = env.reset()

# Initialize an empty list to store all states
all_states = []

# Collect a bunch of states from the environment
for _ in range(2000):  # You can adjust the number of samples as needed
    action = random.randrange(env.num_actions())
    # print(env.step(action))
    _, done = env.act(action)
    state = env.state().reshape(-1,)
    all_states.append(state)
    if done:
        state = env.reset()

# Convert the list of states into a numpy array
all_states = np.array(all_states)

# Perform K-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(all_states)
# Get the indices of states belonging to each cluster
cluster_indices = [np.where(kmeans.labels_ == i)[0] for i in range(k)]

# Sample a random state from each cluster
# print(cluster_indices)
# Print the sampled states from each cluster
# for i, sampled_state in enumerate(clustered_states):
#     print(f"Sampled state from cluster {i+1}: {sampled_state}")


/home/scl/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [8]:
import math
def call_correlation_coeff_kmeans():

    state_vector = []
    for indices in cluster_indices:
    # Randomly select an index from the cluster
      sampled_index = np.random.choice(indices)
      # Get the corresponding state
      sampled_state = get_state(all_states[sampled_index].reshape(env.state().shape))
      # Add the sampled state to the list
      state_vector.append(policy_net.forward_correlation(sampled_state))
      # state_vector.append(sampled_state)

    stacked_tensor = torch.stack(state_vector)
    reshaped_tensor = stacked_tensor.reshape(len(state_vector), -1)
    correlation_matrix = torch.corrcoef(reshaped_tensor)
#     print(correlation_matrix)
    correlation_values = torch.masked_select(correlation_matrix, torch.triu(torch.ones_like(correlation_matrix), diagonal=1).bool())
    # print(state_vector)
    # Average correlation across all pairs of tensor vectors
    average_correlation = correlation_values.mean()
    if math.isnan(average_correlation): 
#         print("nan-correlation")
        return 0
    return average_correlation

In [9]:
call_correlation_coeff_kmeans()

tensor(0.6194, device='cuda:0', grad_fn=<MeanBackward0>)

In [10]:
import pickle
from torch.autograd import Variable
global corr_mult 
corr_mult =1

def optimize_model():
    if not hasattr(optimize_model, "count"):
        optimize_model.count = 0

    optimize_model.count += 1



    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    # print("works until here")
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    # print(state_batch[0])
    # print("policy"  + str(state_action_values[0]))


    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    # print(reward_batch)
    # print("expected" + str(expected_state_action_values[0]))

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    # print(loss)
    # if (optimize_model.count % 60 == 0): print(loss)
    # print(loss)

    # loss = Variable(loss, requires_grad = True)

    # with torch.no_grad():
    correlation_coeff = call_correlation_coeff_kmeans()
    # print(correlation_coeff)

    # Define regularization strength

    # For LUNAAAA
    lambda_corr = 0.1 # You can adjust this parameter as needed

    #for ACROOOO
    lambda_corr = 0.1
#     print(loss, correlation_coeff)

     #for Mountyainnnnn
    
#     corr_mult = 1
    lambda_corr = 1 - corr_mult
#     lambda_corr = 0.1
#     print(loss, correlation_coeff)
#     print(lambda_corr)
    # Add correlation coefficient regularization term to the loss
    loss += (min(0.01,lambda_corr) * correlation_coeff) #(lambda_corr* correlation_coeff)
#     print(loss)
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
#     torch.nn.utils.clip_grad_value_(policy_net.parameters(), 1)
    optimizer.step()
    # return correlation_coeff

In [11]:
# # For plain dqn
# import pickle
# from torch.autograd import Variable
# global corr_mult 
# corr_mult =1
# def optimize_model():
#     if not hasattr(optimize_model, "count"):
#         optimize_model.count = 0

#     optimize_model.count += 1



#     if len(memory) < BATCH_SIZE:
#         return
#     transitions = memory.sample(BATCH_SIZE)
#     # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
#     # detailed explanation). This converts batch-array of Transitions
#     # to Transition of batch-arrays.
#     batch = Transition(*zip(*transitions))

#     # Compute a mask of non-final states and concatenate the batch elements
#     # (a final state would've been the one after which simulation ended)
#     non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
#                                           batch.next_state)), device=device, dtype=torch.bool)
#     non_final_next_states = torch.cat([s for s in batch.next_state
#                                                 if s is not None])
#     state_batch = torch.cat(batch.state)
#     action_batch = torch.cat(batch.action)
#     reward_batch = torch.cat(batch.reward)

#     # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
#     # columns of actions taken. These are the actions which would've been taken
#     # for each batch state according to policy_net
#     # print("works until here")
#     state_action_values = policy_net(state_batch).gather(1, action_batch)
#     # print(state_batch[0])
#     # print("policy"  + str(state_action_values[0]))


#     # Compute V(s_{t+1}) for all next states.
#     # Expected values of actions for non_final_next_states are computed based
#     # on the "older" target_net; selecting their best reward with max(1).values
#     # This is merged based on the mask, such that we'll have either the expected
#     # state value or 0 in case the state was final.
#     next_state_values = torch.zeros(BATCH_SIZE, device=device)
#     with torch.no_grad():
#         next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
#     # Compute the expected Q values
#     expected_state_action_values = (next_state_values * GAMMA) + reward_batch
#     # print(reward_batch)
#     # print("expected" + str(expected_state_action_values[0]))

#     # Compute Huber loss
#     criterion = nn.SmoothL1Loss()
#     loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

#     correlation_coeff = call_correlation_coeff_kmeans()
#     # print(loss)
#     # if (optimize_model.count % 60 == 0): print(loss)
#     # print(loss)

#     # loss = Variable(loss, requires_grad = True)

    
# #     print(loss)
#     # Optimize the model
#     optimizer.zero_grad()
#     loss.backward()
#     # In-place gradient clipping
# #     torch.nn.utils.clip_grad_value_(policy_net.parameters(), 1)
#     optimizer.step()
#     return correlation_coeff

In [12]:
if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 500
    
num_episodes = 1000000

for i_episode in range(num_episodes):
    # if((i_episode + 1) % 25 == 0):
    #     with open("train_dqn_" + str(i_episode + 1) + ".pkl", 'wb') as file:
    # # Serialize and write the object to the file
    #         pickle.dump(policy_net, file)

    # Initialize the environment and get its state
    env.reset()
    state = get_state(env.state())
#     state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    tot_rew = 0

    for t in count():
        # print(t)
        action = select_action(state)
        # print(action)
        # Act according to the action and observe the transition and reward
        reward, done = env.act(action)
        observation = get_state(env.state())
        # if (np.mean(episode_durations[-10:]) >= 25) :
            # env.display_state(500)
            # env.close_display()
        
#         observation, reward, done, info, _= env.step(action.item())
        # print(rew)

        #For mountain car HEREEEE
        # state_0 = state[0][0].item()
        # reward = 50 if observation[0]>=0.5 else abs(observation[0] - state_0)*100
        # print(reward)
        
        tot_rew += reward
        
        reward = torch.tensor([reward], device=device)

        # try gamma later
        
        # print(tot_rew)

        

        if done:
            next_state = None
        else:
            next_state = (observation)

#         done = done or info
        
        # Store the transition in memory
        memory.push(state, action, next_state, reward)
        # Move to the next state
#         print(policy_net.forward(state))
        state = next_state

        # Perform one step of the optimization (on the policy network)

        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(tot_rew)
            corr_mult *= 0.999
            if i_episode > 1:# and np.remainder(i_episode,100) == 0:
                print(i_episode,call_correlation_coeff_kmeans(),tot_rew,1- corr_mult,np.mean(episode_durations[-10:]))
            # plot_durations()
#                 print(i_episode, tot_rew)
            break
    
    if np.mean(episode_durations[-10:]) > 5:
        break
    
env.close_display()
# print('Complete')
# plot_durations(show_result=True)
# plt.ioff()
# plt.show()

2 tensor(0.2389, device='cuda:0', grad_fn=<MeanBackward0>) 1 0.002997000999999999 1.3333333333333333
3 tensor(0.0936, device='cuda:0', grad_fn=<MeanBackward0>) 1 0.003994003998999962 1.25
4 tensor(0.0680, device='cuda:0', grad_fn=<MeanBackward0>) 0 0.004990009995000988 1.0
5 tensor(0.0113, device='cuda:0', grad_fn=<MeanBackward0>) 1 0.0059850199850060015 1.0
6 tensor(0.0141, device='cuda:0', grad_fn=<MeanBackward0>) 0 0.006979034965020947 0.8571428571428571
7 tensor(4.3747e-05, device='cuda:0', grad_fn=<MeanBackward0>) 0 0.007972055930055899 0.75
8 tensor(-0.0049, device='cuda:0', grad_fn=<MeanBackward0>) 0 0.008964083874125839 0.6666666666666666
9 tensor(-0.0051, device='cuda:0', grad_fn=<MeanBackward0>) 1 0.009955119790251765 0.7
10 tensor(-0.0037, device='cuda:0', grad_fn=<MeanBackward0>) 0 0.010945164670461471 0.6
11 tensor(0.0026, device='cuda:0', grad_fn=<MeanBackward0>) 1 0.011934219505790988 0.5
12 tensor(-0.0193, device='cuda:0', grad_fn=<MeanBackward0>) 0 0.012922285286285251

# Noisy evaluation

In [13]:
# num_episodes = 100
# episode_durations = []
# for i_episode in range(num_episodes):
#     # if((i_episode + 1) % 25 == 0):
#     #     with open("train_dqn_" + str(i_episode + 1) + ".pkl", 'wb') as file:
#     # # Serialize and write the object to the file
#     #         pickle.dump(policy_net, file)

#     # Initialize the environment and get its state
#     state, info = env.reset()
#     state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
#     tot_rew = 0

#     for t in count():
#         # print(t)
#         state += torch.randn(state.size())*1
#         action = select_action(state)
#         # print(action)
#         observation, reward, done, info, _= env.step(action.item())
#         # print(rew)

#         #For mountain car HEREEEE
#         # state_0 = state[0][0].item()
#         # reward = 50 if observation[0]>=0.5 else abs(observation[0] - state_0)*100
#         # print(reward)

#         tot_rew += reward
        
#         reward = torch.tensor([reward], device=device)

#         # try gamma later
        
#         # print(tot_rew)

        

#         if done:
#             next_state = None
#         else:
#             next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

#         done = done or info
        
#         # Store the transition in memory
# #         memory.push(state, action, next_state, reward)

#         # Move to the next state
#         state = next_state

#         # Perform one step of the optimization (on the policy network)

# #         optimize_model()

#         # Soft update of the target network's weights
#         # θ′ ← τ θ + (1 −τ )θ′
# #         target_net_state_dict = target_net.state_dict()
# #         policy_net_state_dict = policy_net.state_dict()
# #         for key in policy_net_state_dict:
# #             target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
# #         target_net.load_state_dict(target_net_state_dict)

#         if done:
#             episode_durations.append(tot_rew)
# #             if i_episode > 1:# and np.remainder(i_episode,100) == 0:
#             print(i_episode,call_correlation_coeff_kmeans(),tot_rew)
#             # plot_durations()
#             break
# print(np.mean(episode_durations))

# # print('Complete')
# # plot_durations(show_result=True)
# # plt.ioff()
# # plt.show()

In [14]:
# import pickle
# with open("dqn_lr_-4.pkl", 'wb') as file:
#     # Serialize and write the object to the file
#     pickle.dump(policy_net, file)

In [15]:
# import pickle
# with open("/kaggle/input/pickle-file/corr_dqn.pkl", 'rb') as file:
#     # Serialize and write the object to the file
#     policy_net = pickle.load(file)

In [16]:
# import pickle
# with open("dqn.pkl", 'wb') as file:
#     # Serialize and write the object to the file
#     pickle.dump(policy_net, file)

In [17]:
# import pickle
# with open("dqn_plain.pkl", 'wb') as file:
#     # Serialize and write the object to the file
#     pickle.dump(policy_net, file)

In [18]:
# import pickle
# with open("dqn_plain.pkl", 'rb') as file:
#     # Serialize and write the object to the file
#     policy_net = pickle.load(file)

In [19]:
# import pickle
# with open("dqn.pkl", 'rb') as file:
#     # Serialize and write the object to the file
#     policy_net = pickle.load(file)

In [20]:
def phi_wrapper(state, marker = 0):
    if(marker == 1): state = state[0]
    # print("hello" + str(state.shape))
    state = get_state(state)
    x = policy_net.forward_correlation((state)).detach().cpu().numpy()
    return x #x/np.linalg.norm(x)

In [21]:
# class phi(nn.Module):

#     def __init__(self, dqn):
#         super(phi, self).__init__()
#         self.layer1 = dqn.layer1
#         self.layer2 = dqn.layer2
# #         self.layer3 = dqn.layer3
#         # self.layer4 = dqn.layer4

#         # self.layer3 = nn.Linear(128, n_actions)

#     # Called with either one element to determine next action, or a batch
#     # during optimization. Returns tensor([[left0exp,right0exp]...]).
#     def forward(self, x):         
#         x = torch.tensor(x)
#         x = F.relu(self.layer1(x))
# #         x = F.relu(self.layer2(x))
#         # x = F.relu(self.layer3(x))
#         # x = F.relu(self.layer4(x))
#         # x = F.relu(self.layer2(x))
#         return F.relu(self.layer2(x))
#     def forward_correlation(self, x):
#         x = F.relu(self.layer1(x))
# #         x = F.relu(self.layer2(x))
# #         x = F.relu(self.layer3(x))
#         # x = F.relu(self.layer4(x))
#         return F.relu(self.layer2(x))

In [22]:
# trained_phi = phi(policy_net)


In [23]:
# import pickle
# with open("/kaggle/working/corr_dqn.pkl", "wb") as file:
#     pickle.dump(policy_net, file)

In [24]:
# import pickle
# with open("trained_phi.pkl", 'wb') as file:
#     # Serialize and write the object to the file
#     pickle.dump(trained_phi, file)

In [25]:
# import pickle
# with open("/kaggle/working/trained_phi.pkl", 'rb') as file:
#     # Serialize and write the object to the file
#     policy_net = pickle.load(file)

In [26]:
num_features = 128  # Number of features (state dimensions)
num_actions = env.num_actions()  # Number of actions
weights = np.zeros((num_features, num_actions))


# new corrected code for script

In [27]:
# import gym
import numpy as np
# import copy

# Hyperparameters
alpha = 0.001  # Learning rate, 0.001 for mountaincar
gamma = 0.99 # Discount factor
  # Epsilon-greedy exploration parameter
num_episodes = 30000# Number of episodes
# steps_compl = 0
# Create the CartPole environment
# env = gym.make('CartPole-v1', render_mode = 'none')
# env = gym.make('LunarLander-v2', render_mode = 'none')
# Initialize the weights for the linear function approximation

# weights = np.random.rand(num_features, num_actions)
# weights = copy.deepcopy(curr_nn_weights)

# print("hskgd")


# Linear function approximation
def approximate_q(state, weights):
    return np.dot(phi_wrapper(state), weights).reshape(-1,1)

# curr_test_state = env.observation_space.sample()
# print("hello", (approximate_q(curr_test_state), curr_policy_net(torch.tensor(curr_test_state)).detach().numpy()))

# Epsilon-greedy policy
def epsilon_greedy_policy(state,tot_steps,weights):
    epsilon = 0.1 + math.exp(-1. * tot_steps / 5000)
    if np.random.rand() < epsilon:
        return random.randrange(env.num_actions())  # Random action
    else:
        q_values = approximate_q(state,weights)
        return np.argmax(q_values)  # Greedy action

# Q-learning algorithm with linear function approximation
# env = gym.make(
#     "LunarLander-v2",
#     continuous = False,
#     gravity = -10.0,
#     enable_wind = False,
#     wind_power = 15.0,
#     turbulence_power = 1.5,
#     render_mode = 'human'
# )
res = []
for i in range(10):
    weights = np.zeros((num_features, num_actions))
    duration_list = []
    tot_steps = 0 
    for episode in range(num_episodes):

        env.reset()
        
        state = env.state()
#         state = state[0]
#         state_length = state.size
        # print(len(state))
        done = False
        steps = 0
        # episode_duration  = 0
        tot_rew = 0
        while not done:
    #         env.render()
            steps +=1
            tot_steps +=1
            # episode_duration += 1
    #         state += np.random.normal(0,0.1,state_length)
            action = epsilon_greedy_policy(state,tot_steps,weights)
            # print(action)
            # print(action)
            # print((env.step(action)))
            # env.close_display()
            reward, done = env.act(action)
            next_state = (env.state())
            # env.display_state(500)
            # env.close_display()  
#             next_state, reward, done, _, _ = env.step(action)
    #         if done == False:
    #             reward += 1 
            tot_rew += reward

            # print(state)
            # Update Q-value using the Q-learning update rule
            # print((next_state).shape)
            # print(np.max(approximate_q((next_state))).shape)
            # print(next_state)
            if done:
                td_target = reward
            else:
                td_target = reward + gamma * np.max(approximate_q(next_state,weights))
            td_error = td_target - approximate_q(state,weights)[action]
#             print(td_error)
    #         print(td_target, td_error)
            weights[:, action] += alpha * td_error * phi_wrapper(state).reshape(num_features,)
            # print(nn_weights)
            # print(alpha * td_error * phi_wrapper(state))

            state = next_state

            # if steps >= 1000:
            #     break
        duration_list.append(tot_rew)
    #     plt.plot(duration_list)
    #     plt.pause(0.001)  # pause a bit so that plots are updated
    #     display.display(plt.gcf())
    #     display.clear_output(wait=True)
#         print(tot_rew)
        if(episode % 10 == 0):
            print(np.mean(duration_list[-10:]), tot_rew)
        if np.mean(duration_list[-10:]) > 5:
            print("exit at", i, episode)
            res.append(episode)
            break
    
    else:
        res.append(num_episodes)
  
        
        
print(res)
print(sum(res)/len(res))
    # if (episode + 1) % 100 == 0:
#     print(f"Episode {episode + 1}/{num_episodes}")

# plt.pause(0.001)  # pause a bit so that plots are updated
# display.display(plt.gcf())

# plt.show()

# Close the environment
# env.close()

0.0 0
0.6 2
0.4 0
0.3 0
0.5 0


0.8 0
0.9 1
0.4 0
0.1 1
0.4 0
0.7 1
0.1 0
0.2 0
0.6 2
0.4 1
0.7 0
0.4 0
0.6 0
0.1 0
0.9 0
0.3 0
0.6 0
0.1 0
0.2 0
1.5 1
1.4 3
0.9 1
0.7 0
0.0 0
0.7 0
0.4 0
0.5 0
0.5 0
0.5 2
0.6 0
0.4 0
0.8 2
0.6 0
0.6 3
0.9 0
0.5 0
1.3 1
0.2 1
1.0 2
1.0 0
1.4 0
0.5 0
0.9 0
1.5 1
0.2 0
0.9 0
0.5 0
0.9 0
2.1 2
1.6 0
1.4 7
0.8 0
0.5 0
1.9 1
1.3 1
1.2 0
0.8 0
1.6 2
1.6 3
0.9 0
0.9 0
0.6 0
0.5 1
1.1 0
1.0 0
0.5 0
1.1 0
1.2 0
0.9 1
1.6 0
1.0 3
0.7 1
1.8 1
0.8 0
1.3 0
1.1 1
1.2 0
0.7 0
0.5 0
1.5 2
1.5 3
1.9 1
1.4 0
1.4 0
0.4 0
0.4 0
1.1 2
0.7 0
1.1 0
0.9 7
0.9 1
1.8 2
0.9 0
1.0 2
2.6 2
0.1 0
2.3 5
0.9 0
0.5 0
1.4 3
1.7 1
1.3 2
0.7 0
2.1 0
1.0 1
1.7 6
0.8 0
0.9 0
0.9 0
1.8 0
0.9 0
1.1 4
0.7 0
1.0 0
2.1 0
1.1 1
2.3 1
0.8 0
0.8 2
0.6 0
2.7 5
0.7 0
1.0 2
1.0 0
1.0 0
1.3 0
1.7 5
1.3 1
1.2 0
0.9 3
0.4 0
1.7 0
1.5 0
2.0 1
1.5 1
0.8 0
0.9 2
1.8 2
1.6 0
1.0 3
0.8 1
0.9 2
0.7 1
1.7 1
0.8 1
0.7 2
1.2 2
2.0 2
1.3 1
0.9 0
2.4 0
0.7 0
1.3 0
1.1 0
2.5 0
0.7 0
1.5 0
1.4 0
1.1 0
0.8 0
1.0 0
0.6 0
1.3 0
0.7 0
1.3 1
0.9 0
1.3 

In [28]:
# # import gym
# import numpy as np
# import copy

# # Hyperparameters
# alpha = 0.00001  # Learning rate, 0.001 for mountaincar
# gamma = 0.99  # Discount factor
#   # Epsilon-greedy exploration parameter
# num_episodes = 10000# Number of episodes
# steps_compl = 0
# # Create the CartPole environment
# # env = gym.make('CartPole-v1', render_mode = 'none')
# # env = gym.make('LunarLander-v2', render_mode = 'none')
# # Initialize the weights for the linear function approximation

# # weights = np.random.rand(num_features, num_actions)
# # weights = copy.deepcopy(curr_nn_weights)

# # print("hskgd")


# # Linear function approximation
# def approximate_q(state, weights = weights):
#     # state = state[0]
#     # print(np.array(state).shape)
#     # print(phi_wrapper(state).shape)
#     # print(phi_wrapper(state).shape)
#     # print(weights.shape)
#     return np.dot(phi_wrapper(state), weights)

# curr_test_state = env.observation_space.sample()
# # print("hello", (approximate_q(curr_test_state), curr_policy_net(torch.tensor(curr_test_state)).detach().numpy()))

# # Epsilon-greedy policy
# def epsilon_greedy_policy(state):
#     global steps_compl
#     epsilon = 0.1 + math.exp(-1. * steps_compl / 5000)
#     # print(epsilon)
# #     epsilon = 0.3
# #     epsilon = 0.3
#     steps_compl += 1
#     if np.random.rand() < epsilon:
#         return env.action_space.sample()  # Random action
#     else:
#         q_values = approximate_q(state)
#         # print(q_values.shape)
#         return np.argmax(q_values)  # Greedy action

# # Q-learning algorithm with linear function approximation
# # env = gym.make(
# #     "LunarLander-v2",
# #     continuous = False,
# #     gravity = -10.0,
# #     enable_wind = False,
# #     wind_power = 15.0,
# #     turbulence_power = 1.5,
# #     render_mode = 'human'
# # )
# # for i in range(2):
# duration_list = []
# #     weights = np.zeros((num_features, num_actions))
# #     steps_compl = 0
# for episode in range(num_episodes):

#     state = env.reset()
#     state = state[0]
#     state_length = state.size
#     # print(len(state))
#     done = False
#     steps = 0
#     # episode_duration  = 0
#     tot_rew = 0
#     while not done:
# #         env.render()
#         steps +=1
#         # episode_duration += 1
# #         state += np.random.normal(0,0.1,state_length)
#         action = epsilon_greedy_policy(state)
#         # print(action)
#         # print(action)
#         # print((env.step(action)))
#         next_state, reward, done, _, _ = env.step(action)
# #         if done == False:
# #             reward = 0
#         tot_rew += reward

#     # print(state)
#     # Update Q-value using the Q-learning update rule
#     # print((next_state).shape)
#     # print(np.max(approximate_q((next_state))).shape)
#     # print(next_state)
#         if done:
#             td_target = reward
#         else:
#             td_target = reward + gamma * np.max(approximate_q((next_state)))
#         td_error = td_target - approximate_q((state))[action]
# #         print(td_target, td_error)
#         weights[:, action] += alpha * td_error * phi_wrapper(state)
#         # print(nn_weights)
#         # print(alpha * td_error * phi_wrapper(state))

#         state = next_state
#         if steps >= 1000:
#             break
#     duration_list.append(tot_rew)
# #     plt.plot(duration_list)
# #     plt.pause(0.001)  # pause a bit so that plots are updated
# #     display.display(plt.gcf())
# #     display.clear_output(wait=True)
#     if episode%100==0: print(tot_rew)
#     if np.mean(duration_list[-10:]) > 200:
#         print("exit at", episode)
#         break
#     # if (episode + 1) % 100 == 0:
# #     print(f"Episode {episode + 1}/{num_episodes}")

# # plt.pause(0.001)  # pause a bit so that plots are updated
# # display.display(plt.gcf())

# # plt.show()

# # Close the environment
# # env.close()

In [29]:
# # import gym
# import numpy as np
# import copy

# # Hyperparameters
# alpha = 0.00001  # Learning rate, 0.001 for mountaincar
# gamma = 0.99  # Discount factor
#   # Epsilon-greedy exploration parameter
# num_episodes = 10000# Number of episodes
# steps_compl = 0
# # Create the CartPole environment
# # env = gym.make('CartPole-v1', render_mode = 'none')

# # Initialize the weights for the linear function approximation
# num_features = 32  # Number of features (state dimensions)
# num_actions = env.action_space.n  # Number of actions
# # weights = np.random.rand(num_features, num_actions)
# # weights = copy.deepcopy(curr_nn_weights)
# # weights = np.zeros((num_features, num_actions))

# # print("hskgd")


# # Linear function approximation
# def approximate_q(state, weights = weights):
#     # state = state[0]
#     # print(np.array(state).shape)
#     # print(phi_wrapper(state).shape)
#     # print(phi_wrapper(state).shape)
#     # print(weights.shape)
#     return np.dot(phi_wrapper(state), weights)

# curr_test_state = env.observation_space.sample()
# # print("hello", (approximate_q(curr_test_state), curr_policy_net(torch.tensor(curr_test_state)).detach().numpy()))

# # Epsilon-greedy policy
# def epsilon_greedy_policy(state):
#     global steps_compl
#     epsilon = 0.1 + math.exp(-1. * steps_compl / 5000)
# #     print(epsilon)
# #     epsilon = 0.3
#     epsilon = 0
#     steps_compl += 1
#     if np.random.rand() < epsilon:
#         return env.action_space.sample()  # Random action
#     else:
#         q_values = approximate_q(state)
#         # print(q_values.shape)
#         return np.argmax(q_values)  # Greedy action

# # Q-learning algorithm with linear function approximation
# # env = gym.make(
# #     "LunarLander-v2",
# #     continuous = False,
# #     gravity = -10.0,
# #     enable_wind = False,
# #     wind_power = 15.0,
# #     turbulence_power = 1.5,
# #     render_mode = 'human'
# # )
# duration_list = []
# for episode in range(num_episodes):
    
#     state = env.reset()
#     state = state[0]
#     state_length = state.size
#     # print(len(state))
#     done = False
#     steps = 0
#     # episode_duration  = 0
#     tot_rew = 0
#     while not done:
# #         env.render()
#         steps +=1
#         # episode_duration += 1
# #         state += np.random.normal(0,0.1,state_length)
#         action = epsilon_greedy_policy(state)
#         # print(action)
#         # print(action)
#         # print((env.step(action)))
#         next_state, reward, done, _, _ = env.step(action)
#         tot_rew += reward

#         # print(state)
#         # Update Q-value using the Q-learning update rule
#         # print((next_state).shape)
#         # print(np.max(approximate_q((next_state))).shape)
#         # print(next_state)
#         if done:
#             td_target = reward
#         else:
#             td_target = reward + gamma * np.max(approximate_q((next_state)))
#         td_error = td_target - approximate_q((state))[action]
# #         print(td_target, td_error)
#         weights[:, action] += alpha * td_error * phi_wrapper(state)
#         # print(nn_weights)
#         # print(alpha * td_error * phi_wrapper(state))
        
#         state = next_state
#         if steps >= 1000:
#             break
#     duration_list.append(tot_rew)
# #     plt.plot(duration_list)
# #     plt.pause(0.001)  # pause a bit so that plots are updated
# #     display.display(plt.gcf())
# #     display.clear_output(wait=True)
#     print(tot_rew)
#     # if (episode + 1) % 100 == 0:
# #     print(f"Episode {episode + 1}/{num_episodes}")

# # plt.pause(0.001)  # pause a bit so that plots are updated
# # display.display(plt.gcf())

# # plt.show()
# print("The mean is", np.mean(duration_list),np.std(duration_list))
# # Close the environment
# # env.close()